# Markovkjeder

**Spør hva de er vandt med, og bytt til neste år om de gjør motsatt!!**

In [ ]:
def test_stochastic_matrix(M, tol=1e-12):
    for j in range(len(M)):
        assert abs(np.sum(M[:,j]) - 1) < tol, f"column {j} does not sum up to 1."

In [ ]:
def markov_chain(p0, M, N):
    p = np.zeros((N, len(p0)))
    p[0] = p0
    for n in range(N-1):
        p[n+1] = M@p[n]
    return p

## Random Walk

In [ ]:
# left edge
Mx[0][0] = 0.5
Mx[1][0] = 0.5

# right edge
Mx[-2][-1] = 0.5
Mx[-1][-1] = 0.5

# middle part
for i in range(1, Sx-1):
    Mx[i-1][i] = 0.5
    Mx[i+1][i] = 0.5

In [ ]:
p0 = np.zeros(Sx)
p0[x0 - x_min] = 1

In [ ]:
for n in range(Nx-1):
    x[n+1] = x[n] + np.random.choice([-1, 1], size=walkers)
    x[n+1][x[n+1] < x_min] = x_min
    x[n+1][x[n+1] > x_max] = x_max

## Ehrenfest eksperimentet

In [ ]:
def ehrenfest(u0, B, N):
    U = np.zeros(N, dtype=int)
    U[0] = u0
    for n in range(N-1):
        ball = np.random.randint(B)
        if ball < U[n]:  
            U[n+1] = U[n] - 1
        else:
            U[n+1] = U[n] + 1
    return U

In [ ]:
M_E[1, 0] = 1
M_E[B-1, B] = 1
for i in range(1, B):
    M_E[i-1, i] = i/B
    M_E[i+1, i] = 1 - i/B

## Reaksjonskinetikk

### Første-ordens reaksjon

In [ ]:
M[0, 0] = 1 - k1
M[1, 0] = k1
M[1, 1] = 1

In [ ]:
for n in range(Nt_1-1):
    A[n+1] = A[n]
    B[n+1] = B[n]
    for molecule in range(A[n]):
        if np.random.random() < k1:
            A[n+1] -= 1
            B[n+1] += 1

### Konkurrerende reaksjoner

In [ ]:
M[0, 0] = 1 - k1 - k2
M[1, 0] = k1
M[1, 1] = 1
M[2, 0] = k2
M[2, 2] = 1

In [ ]:
ABC = ["A", "B", "C"]
prob = [1 - k1 - k2, k1, k2]
for n in range(Nt_1 - 1):
    X = np.random.choice(ABC, p=prob, size=A[n])
    A[n+1] = np.sum(X == "A")
    B[n+1] = B[n] + np.sum(X == "B")
    C[n+1] = C[n] + np.sum(X == "C")

### Rekker med reaksjoner

In [ ]:
M[0, 0] = 1 - k1
M[1, 0] = k1
M[1, 1] = 1 - k2
M[2, 1] = k2
M[2, 2] = 1

In [ ]:
for n in range(Nt_2-1):
    rA = np.random.random(A[n])
    rB = np.random.random(B[n])
    A_to_B = np.sum(rA < k1)
    B_to_C = np.sum(rB < k2)
    A[n+1] = A[n] - A_to_B
    B[n+1] = B[n] + A_to_B - B_to_C
    C[n+1] = C[n] + B_to_C

### Reaksjoner med likevekt

In [ ]:
M[0, 0] = 1 - k1
M[0, 1] = k_1
M[1, 0] = k1
M[1, 1] = 1 - k2 - k_1
M[2, 1] = k2
M[2, 2] = 1

In [ ]:
for n in range(Nt_2-1):
    pA = np.random.random(A[n])
    pB = np.random.random(B[n])
    A_to_B = np.sum(pA < k1)
    B_to_C = np.sum(pB < k2)
    B_to_A = np.sum(pB < (k_1 + k2)) - B_to_C
    A[n+1] = A[n] - A_to_B + B_to_A
    B[n+1] = B[n] + A_to_B - B_to_A - B_to_C
    C[n+1] = C[n] + B_to_C